In [1]:
!git init
!git remote add origin https://github.com/louis-thevenet/n7-deep-learning.git
!git pull origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 94 (delta 48), reused 64 (delta 23), pack-reused 0 (from 0)
Unpacking objects: 100% (94/94), 2.47 MiB | 3.98 MiB/s, done.
From https://github.com/louis-thevenet/n7-deep-learning
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


In [2]:
import make_dataset

all_proverbs = make_dataset.load_data()

In [3]:
for key in all_proverbs:
    print(f"{key}: {len(all_proverbs[key])} proverbs")
print(f"Total: {sum(len(all_proverbs[key]) for key in all_proverbs)} proverbs")
total_length = sum(len(p) for proverbs in all_proverbs.values() for p in proverbs)
print(f"Total length: {total_length} characters")

print("\n\nExamples of proverbs:")
key = list(all_proverbs.keys())[0]
for i in range(5):
    print(all_proverbs[key][i])


proverbs_db_only_english.txt: 2208 proverbs
proverbs_db.txt: 34142 proverbs
proverbs_digest.txt: 1000 proverbs
Total: 37350 proverbs
Total length: 1853527 characters


Examples of proverbs:
He that brings good news, knocks hard.
Anger and haste hinder good counsel.
Big thunder, little rain.
Romeo must die in order to save the love.
The point is plain as a pike staff.


In [4]:
selected_proverbs_groups = [
    "proverbs_db.txt",
    "proverbs_digest.txt"
]

proverbs = []
for group in selected_proverbs_groups:
    proverbs.extend(all_proverbs[group])

# Fine Tuning

In [5]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install accelerate
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "facebook/opt-125m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/651 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/251M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

On crée un objet DataSet dans lequel on met nos proverbes non tokenisés.
Puis on les donne aux Tokenizer de opt-125m.

In [7]:
from datasets import Dataset
used_proverbs = proverbs[0:20000]
dataset = Dataset.from_dict({"text": used_proverbs})
print(dataset)
max_length = max([len(x) for x in used_proverbs])
def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Dataset({
    features: ['text'],
    num_rows: 20000
})


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/251M [00:00<?, ?B/s]

In [8]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 294,912 || all params: 125,534,208 || trainable%: 0.2349


In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="no",
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-9-70358b48a957>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,7.181800
20,6.757300
30,6.477900
40,6.232300
50,5.860600
60,5.332700
70,4.527300
80,3.528400
90,2.200100
100,1.182800


TrainOutput(global_step=2500, training_loss=0.3367435408592224, metrics={'train_runtime': 1481.6077, 'train_samples_per_second': 13.499, 'train_steps_per_second': 1.687, 'total_flos': 3799822602240000.0, 'train_loss': 0.3367435408592224, 'epoch': 1.0})

In [17]:
from transformers import pipeline

# Configurer un pipeline de génération
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "A"
results = generator(prompt, max_length=max_length, num_return_sequences=10, do_sample=True, temperature=0.7)

for i, result in enumerate(results):
    print(f"--- Génération {i+1} ---")
    print(result['generated_text'])

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJa

--- Génération 1 ---
A great one must have his own way.
--- Génération 2 ---
A girl should not be forced to marry her fian.
--- Génération 3 ---
A nice dog is a dog.
--- Génération 4 ---
A man who keeps his family safe can never be found.
--- Génération 5 ---
A man who has no life should have a better life.
--- Génération 6 ---
A man who is always in the right will always be in the wrong.
--- Génération 7 ---
A girl that runs from a bear will be a good girl.
--- Génération 8 ---
A good friend is a good friend.
--- Génération 9 ---
A man who will eat a duck.
--- Génération 10 ---
A child that has no teeth is a coward.
